## Fluid dynamic model for PV module temperature estimation

<img src="sourcedata/PV_module_temp.png"><br>
Source: DelftX training; delftx.tudelft.nl

### Calculation of a PV module temperature at given conditions

In [1]:
# Meteorological Variables
Gm = 400 # W/m2
w = 5 # wind sped (m/s)
Tgr = 13+273 # Temperature of ground (Kelvin)
Ta = 15+273 # Temperature of air (Kelvin)

# PV Module Parameters
L = 1.56 # [m] PV module Length   
W = 1.05 # [m] PV module Width
emi_top = 0.84 # PV module top surface emissivity 
emi_bottom = 0.893 # PV module back surface emissivity
Tnoct = 273+43 # NOCT Temperature is 43 degree celcius, obtained from the module spec sheet.
Ref = 0.1 # reflectivity of PV module surface
Eff_stc = 0.20 # efficiecny of the PV module at STC

#Physical Constants
g = 9.8 # [m/s2] 
Pr = 0.71 # Prandle Number
kvis = 14.6e-6 # air viscosity 
k = 0.026 # [W/m*K] thermal conductivity of air  in   
SB = 5.67e-8 # [W*m^-2*K^-4 ] Stefan-Boltzmnan constant 

In [36]:
Tm = 293  # [K] - Initial Temperature of the PV module --> 20+273 = 293 K
for i in range(5):
    # sky temperature (since we have 6 okta it can be assumed to be equal to air temp)
    Tsky = Ta

    # calculation of radiative heat transfer coefficient between module and sky
    h_r_sky = emi_top * SB * (Tm**2 + Tsky**2) * (Tm + Tsky)

    # calculation of radiative heat transfer coefficient between module and ground
    h_r_gr = emi_bottom * SB * (Tm**2 + Tgr**2) * (Tm + Tgr)
    
    # since wind speed is greater than 3 m/s we have turbulent flow for forced convection

    # hydraulic diameter of a PV module
    Dh = 2*L*W/(L+W)

    # Reynold's number
    Re = w*Dh/kvis

    # air density
    ro = 1.225 # [kg/m3]

    # air specific heat
    cv = 713 # [J/kgK]

    h_forced = 0.028*Re**-0.2/(Pr**0.4)*ro*cv*w

    h_f2 = w**0.8
    
    # free convection

    # Grashof number
    Gr = g*1/Tm*(Tm - Ta)*Dh**3/kvis**2

    # Nusselt number
    Nu = 0.21*(Gr * Pr)**0.32

    # free convection HTC
    h_free = Nu*k/Dh
    
    # total convection for front surface
    h_c_top = (h_free**3 + h_f2**3)**(1/3)
    
    # absorptivity 
    alpha = (1 - Ref)*(1 - Eff_stc)
    
    # temperature INOCT for rack mounted version
    Tinoct = Tnoct-3

    # R is the Ratio of actual heat transfer coefficient (back side) to the ideal heat transfer coefficient (top side)
    R = (alpha*Gm-h_c_top*(Tinoct - Ta) - emi_top*SB*(Tinoct**4 - Tsky**4)) / \
    (h_c_top*(Tinoct - Ta) + emi_top*SB*(Tinoct**4 - Tsky**4))
    
    # HTC for bottom side
    h_c_bottom = R * h_c_top
    
    # total HTC
    h_c = h_c_top + h_c_bottom
    
    # module temperature
    Tm = (alpha*Gm + h_c*Ta + h_r_sky*Tsky + h_r_gr*Tgr) / (h_c + h_r_sky + h_r_gr)

Tm - 273

32.16107157867566

#### Introduction of a function that makes the calculations more general and allows to simulate more cases of working conditions of a PV module

In [42]:
def pvModuleTemp(Tm0, Ta, w, method='1', sky='cloudy', inoct='c', n=5):
    '''Function calculating temperature of a PV module using fluid dynamics model
    
    Inputs:
    ---------------
    Tm0: initial temperature of a module [K]
    Ta: ambient air temperature [K]
    w: wind speed [m/s]
    method: method of calculation of htc for forced convection
        '1': exact from Re and Pr numbers
        '2': approximate based on wind speed
    sky: either cloudy (default) or clear
    inoct: type of pv module mounting
        'a': mounted directly on a roof
        'b1': mounted on a roof with a stand-off w = 1 in
        'b2': mounted on a roof with a stand-off w = 3 in
        'b3': mounted on a roof with a stand-off w = 6 in
        'c': rack mounted (default)
    
    Returns:
    ---------------
    Tm: PV module temperature [degC]'''
    
    Tm = Tm0  # [K] - Initial Temperature of the PV module
    
    for i in range(n):
        
        # Sky temperature calculation
        if sky == 'cloudy':
            Tsky = Ta
        elif sky == 'clear':
            Tsky = 0.0552*Ta**(3/2)
        
        # calculation of radiative heat transfer coefficient between module and sky
        h_r_sky = emi_top * SB * (Tm**2 + Tsky**2) * (Tm + Tsky)

        # calculation of radiative heat transfer coefficient between module and ground
        h_r_gr = emi_bottom * SB * (Tm**2 + Tgr**2) * (Tm + Tgr)
        
        # hydraulic diameter of a PV module
        Dh = 2*L*W/(L+W)

        # Reynold's number
        Re = w*Dh/kvis

        # air density
        ro = 1.225 # [kg/m3]

        # air specific heat
        cv = 713 # [J/kgK]
        
        if w > 3:
            # turbulent flow
            if method == '1':
                h_forced = 0.028*Re**-0.2/(Pr**0.4)*ro*cv*w
            else:
                h_forced = w**0.8
        else:
            # laminar flow
            if method == '1':
                h_forced = 0.86*Re**-0.5/(Pr**0.67)*ro*cv*w
            else:
                h_forced = w**0.5
        
        # free convection

        # Grashof number
        Gr = g*1/Tm*(Tm - Ta)*Dh**3/kvis**2

        # Nusselt number
        Nu = 0.21*(Gr * Pr)**0.32

        # free convection HTC
        h_free = Nu*k/Dh
    
        # total convection for front surface
        h_c_top = (h_free**3 + h_forced**3)**(1/3)
        
        # convection for bottom surface
    
        # absorptivity 
        alpha = (1 - Ref)*(1 - Eff_stc)
    
        # temperature INOCT 
        if inoct == 'c':
            # rack mounted version
            Tinoct = Tnoct-3
        elif inoct == 'a':
            # direcly mounted on a roof
            Tinoct = Tnoct+18
        elif inoct =='b1':
            # mounted on a roof with a stand off w=1 in
            Tinoct = Tnoct+11
        elif inoct =='b2':
            # mounted on a roof with a stand off w=3 in
            Tinoct = Tnoct+2
        elif inoct =='b3':
            # mounted on a roof with a stand off w=1 in
            Tinoct = Tnoct-1
        
        # R is the Ratio of actual heat transfer coefficient (back side) to the ideal heat transfer coefficient (top side)
        R = (alpha*Gm-h_c_top*(Tinoct - Ta) - emi_top*SB*(Tinoct**4 - Tsky**4)) / \
        (h_c_top*(Tinoct - Ta) + emi_top*SB*(Tinoct**4 - Tsky**4))
    
        # HTC for bottom side
        h_c_bottom = R * h_c_top
    
        # total HTC
        h_c = h_c_top + h_c_bottom
    
        # module temperature
        Tm = (alpha*Gm + h_c*Ta + h_r_sky*Tsky + h_r_gr*Tgr) / (h_c + h_r_sky + h_r_gr)
    
    return Tm-273

#### Examples of the function usage

In [43]:
# calculation method of htc for forced convection per definition
pvModuleTemp(Tm0=20+273, Ta=Ta, w=w, method='1')

30.499989132177973

In [38]:
# calculation method of htc for forced convection with approximation
pvModuleTemp(Tm0=293, Ta=Ta, w=w, method='2')

32.16107157867566

In [39]:
# checking influence of more interations (default is 5) here 8 iterations are used
pvModuleTemp(Tm0=293, Ta=Ta, w=w, method='2', n=8)

32.16093377524737

In [45]:
# mounting of a module directly on a roof instead of rack mounting (default)
pvModuleTemp(Tm0=20+273, Ta=Ta, w=w, method='1', inoct='a')

34.24728114028227

In [46]:
# clear sky
pvModuleTemp(Tm0=20+273, Ta=Ta, w=w, method='1', sky='clear')

27.276266627361963

In [40]:
# simulation with laminar flow of forced convection
# htc calculated per definition
pvModuleTemp(Tm0=293, Ta=Ta, w=3, method='1')

31.658847931136165

In [41]:
# simulation with laminar flow of forced convection
# htc calculated with approximation
pvModuleTemp(Tm0=293, Ta=Ta, w=3, method='2')

32.4253310494077

### Comparison of forced convection htc coefficients calculation methods

In [29]:
# hydraulic diameter of a PV module
Dh = 2*L*W/(L+W)

# Reynold's number
Re = w*Dh/kvis

# air density
ro = 1.225 # [kg/m3]

# air specific heat
cv = 713 # [J/kgK]

# turbulent flow
#######################
# based on definition
h_tur1 = 0.028*Re**-0.2/(Pr**0.4)*ro*cv*w

# approximation
h_tur2 = w**0.8

# laminar flow
######################
# based on definition
h_lam1 = 0.86*Re**-0.5/(Pr**0.67)*ro*cv*3

# approximation
h_lam2 = 3**0.5

[(h_tur1, h_tur2), (h_lam1, h_lam2)]

[(10.475776106212734, 3.623898318388478),
 (4.323564713552955, 1.7320508075688772)]